In [18]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [19]:
train_fname = '../data/train_transformed_1.csv'
test_fname = '../data/test.csv'
df = pd.read_csv(train_fname, sep=';')
df_test = pd.read_csv(test_fname, sep=';')
f_cat = ['COUNTRY','FISRT_APP_COUNTRY','FISRT_APP_TYPE','LANGUAGE_OF_FILLING',
        'FIRST_CLASSE','TECHNOLOGIE_SECTOR','MAIN_IPC','FISRT_INV_COUNTRY','FISRT_INV_TYPE','SOURCE_CITED_AGE',
        'SOURCE_IDX_ORI','SOURCE_IDX_RAD']
n_samples,n_variables = df.shape[0],len(f_cat)

In [20]:
# On ne se soucie pas des labels représentant les trous pour l'instant.
X_train = df[f_cat]
y_train = df[['VARIABLE_CIBLE']]
X_test = df_test[f_cat]

In [21]:
# On espère que pour chaque colonne, il y a peu de possibilités (éventuellement quelque unes de plus pour les pays), et 
# qu'on pourra donc utiliser efficassement un classifier de Bayes. Or la colonne 'FIRST_CLASS' contient 36460 labels
# différents. On voit mal ce qu'on pourrait en faire... On décide donc de s'en débarasser.
labels = []

for col in f_cat:
    X = set(X_train[col].tolist())
    labels += set([x for x in X if isinstance(x, basestring)])
    
print(len(labels))
print(len(set(X_train['FIRST_CLASSE'].tolist())))

37336
36460


In [24]:
f_cat = ['COUNTRY','FISRT_APP_COUNTRY','FISRT_APP_TYPE','LANGUAGE_OF_FILLING',
        'TECHNOLOGIE_SECTOR','MAIN_IPC','FISRT_INV_COUNTRY','FISRT_INV_TYPE','SOURCE_CITED_AGE',
        'SOURCE_IDX_ORI','SOURCE_IDX_RAD']
n_samples,n_variables = df.shape[0],len(f_cat)
# On ne se soucie pas des labels représentant les trous pour l'instant.
X_train = df[f_cat]
y_train = df[['VARIABLE_CIBLE']]
X_test = df_test[f_cat]

In [29]:
labels = []

for col in f_cat:
    labels += set((col+':'+X_train[col].astype(str)).tolist())
    
print(len(labels))
print(n_samples)
print(n_samples*len(labels))

880
259431
228299280


### Une matrice de taille 200,000x1000 reste un peu grosse, mais il ne faut pas perdre de vu qu'elle sera booléenne et que le classifier de Bayes n'est pas trop gourmand en puissance (même si ça dépend des variantes).

In [26]:
df_transformed = pd.DataFrame(data=np.zeros((n_samples,len(labels))), columns=labels, dtype=bool)

### Visiblement cette matrice est déjà trop grosse pour python... On va enlever donc les features qui générèrent le plus de labels différents jusqu'à ce que ça rentre...

COUNTRY : 91
FISRT_APP_COUNTRY : 128
MAIN_IPC : 626
FISRT_INV_COUNTRY : 136

In [30]:
f_cat = ['COUNTRY','FISRT_APP_COUNTRY','FISRT_APP_TYPE','LANGUAGE_OF_FILLING',
        'TECHNOLOGIE_SECTOR','MAIN_IPC','FISRT_INV_COUNTRY','FISRT_INV_TYPE','SOURCE_CITED_AGE',
        'SOURCE_IDX_ORI','SOURCE_IDX_RAD']
n_samples,n_variables = df.shape[0],len(f_cat)
# On ne se soucie pas des labels représentant les trous pour l'instant.
X_train = df[f_cat]
y_train = df[['VARIABLE_CIBLE']]
X_test = df_test[f_cat]

In [31]:
labels = []

for col in f_cat:
    X = set((col+':'+(X_train[col]).astype(str)).tolist() + (col+':'+(X_test[col]).astype(str)).tolist())
    labels += set([x for x in X if isinstance(x, basestring)])
    
print(len(labels))
print(n_samples)
print(n_samples*len(labels))

1036
259431
268770516


In [32]:
df_transformed_train = pd.DataFrame(data=np.zeros((n_samples,len(labels))), columns=labels, dtype=bool)
df_transformed_test = pd.DataFrame(data=np.zeros((X_test.shape[0],len(labels))), columns=labels, dtype=bool)

In [33]:
for label in labels:
    lab = label.split(':')
    df_transformed_train[label] = X_train[lab[0]] == lab[1]
    df_transformed_test[label] = X_test[lab[0]] == lab[1]

In [34]:
df_transformed_train.to_csv(path_or_buf='../data/bayes_data_train_2.csv', sep=';')
df_transformed_test.to_csv(path_or_buf='../data/bayes_data_test_2.csv', sep=';')

In [17]:
df[['VARIABLE_CIBLE']].to_csv(path_or_buf='../data/y_train_1.csv', sep=';')